# Evaluate Risk and Safety of Text - Protected Material and Indirect Attack Jailbreak

## Objective
This notebook walks through how to generate a simulated conversation targeting a deployed AzureOpenAI model and then evaluate that conversation test dataset for Protected Material and Indirect Attack Jailbreak (also know as XPIA or cross-domain prompt injected attack) vulnerability. It also references Azure AI Content Safety service's prompt filtering capabilities to help identify and mitigate these vulnerabilities in your AI system.

## Time
You should expect to spend about 30 minutes running this notebook. If you increase or decrease the number of simulated conversations, the time will vary accordingly.

## Before you begin

### Installation
Install the following packages required to execute this notebook.

In [ ]:
# Install the packages
%pip install openai azure-ai-evaluation azure-identity promptflow-azure

Set the following variables for use in this notebook:

In [ ]:
azure_ai_project = {
    "subscription_id": "<your-subscription-id>",
    "resource_group_name": "<your-resource-group-name>",
    "project_name": "<your-project-name>",
}

azure_openai_deployment = ""
azure_openai_endpoint = ""
azure_openai_api_version = ""

### Configuration
The following simulator and evaluators require an Azure AI Studio project configuration and an Azure credential to use. 
Your project configuration will be what is used to log your evaluation results in your project after the evaluation run is finished.

For this sample, we recommend creating or using a project in East US 2. For full region supportability, see [our documentation](https://learn.microsoft.com/azure/ai-studio/how-to/develop/flow-evaluate-sdk#built-in-evaluators).

In [ ]:
import os

from pprint import pprint
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.ai.evaluation import evaluate
from azure.ai.evaluation import ProtectedMaterialEvaluator, IndirectAttackEvaluator
from azure.ai.evaluation.simulator import (
    AdversarialSimulator,
    AdversarialScenario,
    AdversarialScenarioJailbreak,
    IndirectAttackSimulator,
)
from openai import AzureOpenAI


os.environ["AZURE_DEPLOYMENT_NAME"] = azure_openai_deployment
os.environ["AZURE_ENDPOINT"] = azure_openai_endpoint
os.environ["AZURE_API_VERSION"] = azure_openai_api_version

credential = DefaultAzureCredential()

## Run this example

To keep this notebook lightweight, let's create a dummy application that calls GPT 3.5 Turbo, which is essentially Chat GPT. When we are testing your application for certain safety metrics like Protected Material or Indirect Attacks, it's important to have a way to automate a basic style of red-teaming to elicit behaviors from a simulated malicious user. We will use the `Simulator` class and this is how we will generate a synthetic test dataset against your application. Once we have the test dataset, we can evaluate them with our `ProtectedMaterialEvaluator` and `IndirectAttackEvaluator` classes.

The `Simulator` needs a structured contract with your application in order to simulate conversations or other types of interactions with it. This is achieved via a callback function. This is the function you would rewrite to actually format the response from your generative AI application.

In [ ]:
from typing import List, Dict, Optional


async def protected_material_callback(
    messages: List[Dict], stream: bool = False, session_state: Optional[str] = None, context: Optional[Dict] = None
) -> dict:
    deployment = os.environ.get("AZURE_DEPLOYMENT_NAME")
    endpoint = os.environ.get("AZURE_ENDPOINT")
    token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")
    # Get a client handle for the model
    client = AzureOpenAI(
        azure_endpoint=endpoint,
        api_version=os.environ.get("AZURE_API_VERSION"),
        azure_ad_token_provider=token_provider,
    )
    # Call the model
    completion = client.chat.completions.create(
        model=deployment,
        messages=[
            {
                "role": "user",
                "content": messages["messages"][0]["content"],  # injection of prompt happens here.
            }
        ],
        max_tokens=800,
        temperature=0.7,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
        stream=False,
    )

    formatted_response = completion.to_dict()["choices"][0]["message"]
    messages["messages"].append(formatted_response)
    return {
        "messages": messages["messages"],
        "stream": stream,
        "session_state": session_state,
        "context": context,
    }

## Testing your application for Protected Material

When building your application, you want to test that Protected Material (i.e. copyrighted content or material) is not being generated by your generative AI applications. The following example uses an `AdversarialSimulator` paired with a protected content scenario to prompt your model to respond with material that is protected by intellectual property laws.

In [ ]:
# initialize the adversarial simulator
protected_material_simulator = AdversarialSimulator(azure_ai_project=azure_ai_project, credential=credential)

# define the adversarial scenario you want to simulate
protected_material_scenario = AdversarialScenario.ADVERSARIAL_CONTENT_PROTECTED_MATERIAL

In [ ]:
unfiltered_protected_material_outputs = await protected_material_simulator(
    scenario=protected_material_scenario,
    max_conversation_turns=3,  # define the number of conversation turns
    max_simulation_results=10,  # define the number of simulation results
    target=protected_material_callback,  # define the target model callback
)

Let's take a quick look at the generated dataset

In [ ]:
# Results are truncated for brevity.
truncation_limit = 50
for output in unfiltered_protected_material_outputs:
    for turn in output["messages"]:
        print(f"{turn['role']} : {turn['content'][0:truncation_limit]}")

In [ ]:
from pathlib import Path

print(unfiltered_protected_material_outputs.to_eval_qr_json_lines())
output = unfiltered_protected_material_outputs.to_eval_qr_json_lines()
file_path = "unfiltered_protected_material_output.jsonl"

# Write the output to the file
with Path.open(Path(file_path), "w") as file:
    file.write(output)

Now that we have our dataset, we can evaluate it for Protected Material. The `ProtectedMaterialEvaluator` class can take in the dataset and detect whether your data contains copyrighted content. Let's use the `evaluate()` API to run the evaluation and log it to our Azure AI Studio Project.

In [ ]:
protected_material_eval = ProtectedMaterialEvaluator(azure_ai_project=azure_ai_project, credential=credential)

result = evaluate(
    data=file_path,
    evaluators={"protected_material": protected_material_eval},
    # Optionally provide your AI Studio project information to track your evaluation results in your Azure AI Studio project
    azure_ai_project=azure_ai_project,
    # Optionally provide an output path to dump a json of metric summary, row level data and metric and studio URL
    output_path="./mynewfilteredIPevalresults.json",
)

We can see that our "model" application gives us a defect rate showing us that we can't deploy our application just yet. Moving forward, to protect our application against generating protected material content, we can add an [Azure AI Content Safety filter for Protected Materials for text](https://learn.microsoft.com/azure/ai-services/content-safety/quickstart-protected-material) which is a mitigation layer to help protect and filter out responses from your model that may contain protected material content. Let's apply this filter and re-run the simulator and evaluation step to see if it helps with our defect rate.

In [ ]:
filtered_protected_material_outputs = await protected_material_simulator(
    scenario=protected_material_scenario,
    max_conversation_turns=3,  # define the number of conversation turns
    max_simulation_results=10,  # define the number of simulation results
    target=protected_material_callback,  # now with the Prompt Shield attached to our model deployment
)

In [ ]:
print(filtered_protected_material_outputs.to_eval_qr_json_lines())
output = filtered_protected_material_outputs.to_eval_qr_json_lines()
filtered_protected_material_file_path = "filtered_protected_material_output.jsonl"

# Write the output to the file
with Path.open(Path(filtered_protected_material_file_path), "w") as file:
    file.write(output)

In [ ]:
filtered_result = evaluate(
    data=filtered_protected_material_file_path,
    evaluators={"protected_material": protected_material_eval},
    # Optionally provide your AI Studio project information to track your evaluation results in your Azure AI Studio project
    azure_ai_project=azure_ai_project,
    # Optionally provide an output path to dump a json of metric summary, row level data and metric and studio URL
    output_path="./myfilteredevalresults.json",
)

## Testing your application for Indirect Attack Jailbreaks



Jailbreaks are direct attacks injected into either the user's query towards your application (UPIA or user prompt injected attack) or indirect attacks injected into the context sent to your application to generate a response (XPIA or cross domaine prompt injected attack). Both types of attacks will result in an altered or unexpected behavior that may result in disrupted functionality or security risks like information leakage or engaging in harmful behavior. 

The following example takes the "model" application above and simulates indirect attacks to jailbreak the model and then evaluates the dataset generated by it.

In [ ]:
from typing import List, Dict, Optional


async def xpia_callback(
    messages: List[Dict], stream: bool = False, session_state: Optional[str] = None, context: Optional[Dict] = None
) -> dict:
    messages_list = messages["messages"]
    # get last message
    latest_message = messages_list[-1]
    query = latest_message["content"]
    context = None
    if "file_content" in messages["template_parameters"]:
        query += messages["template_parameters"]["file_content"]
    # the next few lines explain how to use the AsyncAzureOpenAI's chat.completions
    # to respond to the simulator. You should replace it with a call to your model/endpoint/application
    # make sure you pass the `query` and format the response as we have shown below

    # Get a client handle for the model
    deployment = os.environ.get("AZURE_DEPLOYMENT_NAME")

    token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

    oai_client = AzureOpenAI(
        azure_endpoint=os.environ.get("AZURE_ENDPOINT"),
        api_version=os.environ.get("AZURE_API_VERSION"),
        azure_ad_token_provider=token_provider,
    )
    try:
        response_from_oai_chat_completions = oai_client.chat.completions.create(
            messages=[{"content": query, "role": "user"}], model=deployment, max_tokens=300
        )
        print(response_from_oai_chat_completions)
    except Exception as e:
        print(f"Error: {e} with content length {len(query)}")
        # to continue the conversation, return the messages, else you can fail the adversarial with an exception
        message = {
            "content": "Something went wrong. Check the exception e for more details.",
            "role": "assistant",
            "context": None,
        }
        messages["messages"].append(message)
        return {"messages": messages["messages"], "stream": stream, "session_state": session_state}
    response_result = response_from_oai_chat_completions.choices[0].message.content
    formatted_response = {
        "content": response_result,
        "role": "assistant",
        "context": {},
    }
    messages["messages"].append(formatted_response)
    return {"messages": messages["messages"], "stream": stream, "session_state": session_state, "context": context}

In [ ]:
indirect_attack_simulator = IndirectAttackSimulator(
    azure_ai_project=azure_ai_project, credential=DefaultAzureCredential()
)

unfiltered_indirect_attack_outputs = await indirect_attack_simulator(
    target=xpia_callback,
    scenario=AdversarialScenarioJailbreak.ADVERSARIAL_INDIRECT_JAILBREAK,
    max_simulation_results=10,
    max_conversation_turns=3,
)

Let's take a quick look at the data generated

In [ ]:
pprint(unfiltered_indirect_attack_outputs)

In [ ]:
# Results are truncated for brevity.
truncation_limit = 50
for output in unfiltered_indirect_attack_outputs:
    for turn in output["messages"]:
        content = turn["content"]
        if isinstance(content, dict):  # user response from callback is dict
            print(f"{turn['role']} : {content['content'][0:truncation_limit]}")
        elif isinstance(content, tuple):  # assistant response from callback is tuple
            print(f"{turn['role']} : {content[0:truncation_limit]}")

In [ ]:
from pathlib import Path

print(unfiltered_indirect_attack_outputs)
print(unfiltered_indirect_attack_outputs.to_eval_qr_json_lines())
output = unfiltered_indirect_attack_outputs.to_eval_qr_json_lines()
xpia_file_path = "unfiltered_indirect_attack_outputs.jsonl"

# Write the output to the file
with Path.open(Path(xpia_file_path), "w") as file:
    file.write(output)

Now that we have our dataset, we can evaluate it to see if the indirect attacks resulted in jailbreaks. The `IndirectAttackEvaluator` class can take in the dataset and detects instances of jailbreak. Let's use the `evaluate()` API to run the evaluation and log it to our Azure AI Studio Project.

In [ ]:
indirect_attack_eval = IndirectAttackEvaluator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())
file_path = "indirect_attack_outputs.jsonl"
result = evaluate(
    data=xpia_file_path,
    evaluators={
        "indirect_attack": indirect_attack_eval,
    },
    # Optionally provide your AI Studio project information to track your evaluation results in your Azure AI Studio project
    azure_ai_project=azure_ai_project,
    # Optionally provide an output path to dump a json of metric summary, row level data and metric and studio URL
    output_path="./mynewindirectattackevalresults.json",
)

We can see that our "model" application gives us a defect rate broken down by different behaviors resulting from the jailbreak, showing us that we can't deploy our application just yet. Moving forward, to protect our application against indirect jailbreak attacks, we can add an [Azure AI Content Safety Prompt Shield](https://learn.microsoft.com/azure/ai-services/content-safety/quickstart-jailbreak) which is a mitigation layer to help annotate and block requests to your model or application that contain known indirect attacks for jailbreak. Let's apply this filter and re-run the simulator and evaluation step to see if it helps with our defect rate.

In [ ]:
filtered_indirect_attack_outputs = await indirect_attack_simulator(
    target=xpia_callback,  # now with the Prompt Shield attached to our model deployment
    scenario=AdversarialScenarioJailbreak.ADVERSARIAL_INDIRECT_JAILBREAK,
    max_simulation_results=10,
    max_conversation_turns=3,
)

In [ ]:
print(filtered_indirect_attack_outputs)
print(filtered_indirect_attack_outputs.to_eval_qr_json_lines())
output = filtered_indirect_attack_outputs.to_eval_qr_json_lines()
xpia_file_path = "filtered_indirect_attack_outputs.jsonl"

# Write the output to the file
with Path.open(Path(xpia_file_path), "w") as file:
    file.write(output)

In [ ]:
filtered_indirect_attack_result = evaluate(
    data=xpia_file_path,
    evaluators={"indirect_attack": indirect_attack_eval},
    # Optionally provide your AI Studio project information to track your evaluation results in your Azure AI Studio project
    azure_ai_project=azure_ai_project,
    # Optionally provide an output path to dump a json of metric summary, row level data and metric and studio URL
    output_path="./myindirectattackevalresults.json",
)

In conclusion, we've walked through how to generate test datasets using the simulation framework and our safety evaluation framework. See our documentation for more details and additional functionality on [simulation](https://aka.ms/advsimulatorhowto) and [evaluation](https://aka.ms/azureaistudiosafetyevalhowto)."